<a href="https://colab.research.google.com/github/talpt/pyton/blob/main/Williams_R_ProTrend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Gerekli kütüphaneleri yükleyin
!pip install git+https://github.com/rongardF/tvdatafeed
!pip install tradingview-screener==2.5.0

import pandas as pd
import numpy as np
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

# TvDatafeed nesnesi oluşturma
tv = TvDatafeed()

# Türkiye piyasasındaki tüm sembolleri almak için
try:
    Hisseler = get_all_symbols(market='turkey')
    Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
    Hisseler = sorted(Hisseler)
except Exception as e:
    print(f"Hata alındı: {e}")

# Zaman dilimi seçimi için fonksiyon
def get_interval_choice():
    intervals = {
        '1': ('15 Dakika', Interval.in_15_minute),
        '2': ('30 Dakika', Interval.in_30_minute),
        '3': ('45 Dakika', Interval.in_45_minute),
        '4': ('1 Saat', Interval.in_1_hour),
        '5': ('2 Saat', Interval.in_2_hour),
        '6': ('4 Saat', Interval.in_4_hour),
        '7': ('Günlük', Interval.in_daily),
        '8': ('Haftalık', Interval.in_weekly),
        '9': ('Aylık', Interval.in_monthly)
    }

    print("Lütfen tarama için bir zaman dilimi seçin:")
    for key, (name, _) in intervals.items():
        print(f"{key}: {name}")

    while True:
        choice = input("Seçiminizi yapın (1-9): ")
        if choice in intervals:
            return intervals[choice][1]
        else:
            print("Geçersiz seçim, lütfen tekrar deneyin.")

# Teknik analiz fonksiyonları ve tarama işlemi kodu (mevcut kod ile aynı)
def _pr(src, length):
    highest_high = src.rolling(window=length).max()
    lowest_low = src.rolling(window=length).min()
    wpr = 100 * (src - highest_high) / (highest_high - lowest_low)
    return wpr

def variant(type, src, length):
    if type == "SMA":
        return src.rolling(window=length).mean()
    elif type == "EMA":
        return src.ewm(span=length, adjust=False).mean()
    elif type == "WMA":
        weights = np.arange(1, length + 1)
        return src.rolling(window=length).apply(lambda prices: np.dot(prices, weights) / weights.sum(), raw=True)
    elif type == "RMA":
        return src.ewm(alpha=1/length, adjust=False).mean()
    return src

def bollinger_bands(type, src, length, mult):
    basis = variant(type, src, length)
    dev = mult * src.rolling(window=length).std()
    upper = basis + dev
    lower = basis - dev
    return basis, upper, lower

def calculate_momentum(data, window=14):
    return data.diff(window)

def ichimoku(data):
    data['tenkan_sen'] = (data['high'].rolling(window=9).max() + data['low'].rolling(window=9).min()) / 2
    data['kijun_sen'] = (data['high'].rolling(window=26).max() + data['low'].rolling(window=26).min()) / 2
    data['senkou_span_a'] = ((data['tenkan_sen'] + data['kijun_sen']) / 2).shift(26)
    data['senkou_span_b'] = ((data['high'].rolling(window=52).max() + data['low'].rolling(window=52).min()) / 2).shift(26)
    data['chikou_span'] = data['close'].shift(-26)

def parabolic_sar(data, acceleration=0.02, max_acceleration=0.2):
    sar = np.zeros(len(data))
    sar[0] = data['close'][0]
    is_uptrend = True
    ep = data['low'][0]
    af = acceleration

    for i in range(1, len(data)):
        sar[i] = sar[i - 1] + af * (ep - sar[i - 1])
        if is_uptrend:
            sar[i] = max(sar[i], data['low'][i - 1])
            if data['high'][i] > ep:
                ep = data['high'][i]
                af = min(af + acceleration, max_acceleration)
            if data['low'][i] < sar[i]:
                is_uptrend = False
                sar[i] = ep
                ep = data['low'][i]
                af = acceleration
        else:
            sar[i] = min(sar[i], data['high'][i - 1])
            if data['low'][i] < ep:
                ep = data['low'][i]
                af = min(af + acceleration, max_acceleration)
            if data['high'][i] > sar[i]:
                is_uptrend = True
                sar[i] = ep
                ep = data['high'][i]
                af = acceleration
    data['psar'] = sar

def scan_stocks(symbols, interval, n_bars=500, wpr_period=25, wpr_smoothing_period=3, wpr_smoothing_type="SMA", bb_period=89, bb_mult=1.0, bb_type="SMA", wma_length=50, momentum_window=14):
    results = []

    for symbol in symbols:
        try:
            data = tv.get_hist(symbol=symbol, exchange='BIST', interval=interval, n_bars=n_bars)
            if data is None or data.empty:
                print(f"{symbol} için veri bulunamadı veya boş.")
                continue

            data['wpr'] = (_pr(data['close'], wpr_period) + 50.0) * 2.0
            data['wpr_smoothed'] = variant(wpr_smoothing_type, data['wpr'], wpr_smoothing_period)

            middle, upper, lower = bollinger_bands(bb_type, data['wpr_smoothed'], bb_period, bb_mult)
            data['bb_middle'] = middle
            data['bb_upper'] = upper
            data['bb_lower'] = lower

            data['wma'] = variant("WMA", data['close'], wma_length)
            data['momentum'] = calculate_momentum(data['close'], window=momentum_window)
            ichimoku(data)
            parabolic_sar(data)

            data['go_long'] = (
                (data['wpr_smoothed'] > data['bb_upper']) &
                (data['close'] > data['wma']) &
                (data['momentum'] > 0) &
                (data['close'] > data['psar']) &
                (data['tenkan_sen'] > data['kijun_sen']) &
                (data['close'] > data['senkou_span_a']) &
                (data['close'] > data['senkou_span_b'])
            )
            is_long = data.iloc[-1]['go_long']

            print(f"{symbol}: Long Sinyal: {is_long}")

            if is_long:
                results.append({
                    'Hisse Adı': symbol,
                    'Son Fiyat': data.iloc[-1]['close'],
                    'Sinyal': 'Long'
                })
        except Exception as e:
            print(f"Hata alındı {symbol}: {e}")

    return results

# Kullanıcıdan zaman dilimi seçimi
interval = get_interval_choice()

# Tarama işlemini başlatma
results = scan_stocks(Hisseler, interval)

# Sonuçları gösterme
if results:
    df_results = pd.DataFrame(results)
    print("\nKoşulları sağlayan hisseler:")
    print(df_results)
else:
    print("\nHiçbir hisse tarama koşulunu sağlamıyor.")


  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-mycc6jyd
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-mycc6jyd
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done


Lütfen tarama için bir zaman dilimi seçin:
1: 15 Dakika
2: 30 Dakika
3: 45 Dakika
4: 1 Saat
5: 2 Saat
6: 4 Saat
7: Günlük
8: Haftalık
9: Aylık
Seçiminizi yapın (1-9): 5
A1CAP: Long Sinyal: False
ACSEL: Long Sinyal: False
ADEL: Long Sinyal: False
ADESE: Long Sinyal: False
ADGYO: Long Sinyal: False
AEFES: Long Sinyal: False
AFYON: Long Sinyal: False
AGESA: Long Sinyal: False
AGHOL: Long Sinyal: False
AGROT: Long Sinyal: False
AGYO: Long Sinyal: False
AHGAZ: Long Sinyal: False
AHSGY: Long Sinyal: False
AKBNK: Long Sinyal: False
AKCNS: Long Sinyal: False
AKENR: Long Sinyal: False
AKFGY: Long Sinyal: False
AKFYE: Long Sinyal: False
AKGRT: Long Sinyal: False
AKMGY: Long Sinyal: False
AKSA: Long Sinyal: False
AKSEN: Long Sinyal: False
AKSGY: Long Sinyal: False
AKSUE: Long Sinyal: False
AKYHO: Long Sinyal: False
ALARK: Long Sinyal: False
ALBRK: Long Sinyal: False
ALCAR: Long Sinyal: False
ALCTL: Long Sinyal: False
ALFAS: Long Sinyal: False
ALGYO: Long Sinyal: False
ALKA: Long Sinyal: False
ALK